In [33]:
import numpy as np

np.radians(100)

1.7453292519943295

In [1]:
import os
import geopandas as gpd
from shapely.geometry import Point, box as shapely_box
import simplekml
import numpy as np

def make_shapefile(string, coords, width=0.25, height=0.25):
    
    # Create a folder named 'shapefiles'
    output_folders = ['../data_store/data/shapefiles','./shapefiles']

    # Create a GeoDataFrame with a point for Sodankylä and set the CRS explicitly
    point = gpd.GeoDataFrame(geometry=[Point(coords)], crs="EPSG:4326")

    # Create a square (box) with a given width and height
    bounding_box = shapely_box(point.geometry.x[0] - width/np.cos(np.radians(coords[1])), point.geometry.y[0] - height,
                               point.geometry.x[0] + width/np.cos(np.radians(coords[1])), point.geometry.y[0] + height)

    # Convert the box to a GeoDataFrame
    box_gdf = gpd.GeoDataFrame(geometry=[bounding_box], crs="EPSG:4326")

    # Create a KML object
    kml = simplekml.Kml()

    # Create a polygon from the box geometry
    polygon_coords = [(x, y) for x, y in box_gdf.geometry.apply(lambda geom: list(geom.exterior.coords))[0]]
    polygon = kml.newpolygon(name=f"Box around {string}", outerboundaryis=polygon_coords)

    for output_folder in output_folders:
    
        # Save the KML file in the output folder
        os.makedirs(output_folder, exist_ok=True)
        kml_filename = os.path.join(output_folder, f"{string}.kml")
        kml.save(kml_filename)
    
        print(f"KML file '{kml_filename}' created.")
    return


In [37]:
# import geopandas as gpd
# from shapely.geometry import Point, box as shapely_box
# import simplekml
# import os


# def make_shapefile(string, coords, width=25000, height=25000):
    
#     # Create a folder named 'shapefiles'
#     output_folders = ['../data_store/data/shapefiles','./shapefiles']
#     # output_folder = './shapefiles'

#     # Create a GeoDataFrame with a point for Sodankylä and set the CRS explicitly
#     point = gpd.GeoDataFrame(geometry=[Point(coords)], crs="EPSG:4326")

#     # Reproject to a projected CRS (e.g., UTM)
#     point_utm = point.to_crs(epsg=32635)  # You may need to choose an appropriate EPSG code

#     # Create a square (box) with a given width and height
#     bounding_box = shapely_box(point_utm.geometry.x[0] - width, point_utm.geometry.y[0] - height,
#                                point_utm.geometry.x[0] + width, point_utm.geometry.y[0] + height)

#     # Convert the box to a GeoDataFrame
#     box_gdf = gpd.GeoDataFrame(geometry=[bounding_box], crs=point_utm.crs)

#     # Reproject back to the original CRS
#     box = box_gdf.to_crs(epsg=4326)

#     # Create a subfolder for each string
#     # string_folder = os.path.join(output_folder, f"{string}")
#     # os.makedirs(string_folder, exist_ok=True)

#     # Create a KML object
#     kml = simplekml.Kml()

#     # Create a polygon from the box geometry
#     polygon_coords = [(x, y) for x, y in box.geometry.apply(lambda geom: list(geom.exterior.coords))[0]]
#     polygon = kml.newpolygon(name=f"Box around {string}", outerboundaryis=polygon_coords)

#     for output_folder in output_folders:
#         os.makedirs(output_folder, exist_ok=True)
#         # Save the KML file in the subfolder
#         kml_filename = os.path.join(output_folder, f"{string}.kml")
#         kml.save(kml_filename)

#         print(f"KML file '{kml_filename}' created.")
#     return

In [7]:
import numpy as np

make_shapefile(string = "oregon_2",
               coords = (-121.606, 44.3238),
               width = 0.02,
               height = 0.02)

KML file '../data_store/data/shapefiles/oregon_2.kml' created.
KML file './shapefiles/oregon_2.kml' created.
